# cifar10 softmax nn

## imports

In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import pickle

## data
This is CIFAR-10 dataset. Extraction directions are provided on [this](http://www.cs.toronto.edu/~kriz/cifar.html) website.

In [6]:
def unpickle(file):
    with open(file, "rb") as f:
        data_map = pickle.load(f, encoding="bytes")
    return data_map

In [7]:
data_batch_1 = unpickle("../my_datasets/cifar-10-batches-py/data_batch_1")

In [14]:
data_batch_1[b"data"][0]

array([ 59,  43,  50, ..., 140,  84,  72], dtype=uint8)